# 주소 정리

## library

In [968]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

## 데이터 로드

In [969]:
df = pd.read_csv('2020-22년_급속충전.csv', index_col=0, usecols=['충전소명', '주소', '지역', '시군구'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11665822 entries, 내린천휴게소 양양방향 to 마천동2공영주차장
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   지역      object
 1   시군구     object
 2   주소      object
dtypes: object(3)
memory usage: 356.0+ MB


In [970]:
df = df.reset_index()
df.head()

,충전소명,지역,시군구,주소
0,내린천휴게소 양양방향,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향)
1,내린천휴게소 양양방향,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향)
2,내린천휴게소 양양방향,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향)
3,내린천휴게소 양양방향,강원도,인제군,강원도 인제군 상남면 하남리 339-7(양양방향)
4,한국광기술원,광주광역시,북구,광주광역시 북구 첨단 벤처로 108번길 9


In [971]:
adr_uni = df['주소'].unique()
len(adr_uni)

4364

In [972]:
len(df['충전소명'].unique())

4217

In [1343]:
df2 = df.drop_duplicates(['주소'], keep='first').reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4364 entries, 0 to 4363
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   충전소명    4364 non-null   object
 1   지역      4364 non-null   object
 2   시군구     4364 non-null   object
 3   주소      4364 non-null   object
dtypes: object(4)
memory usage: 136.5+ KB


In [1344]:
len(df2['충전소명'].unique())

4161

In [1345]:
adr = df2['주소'].copy().to_numpy()
len(adr)

4364

In [1346]:
id(adr), id(df2['주소'])

(2590020218768, 2590039609936)

In [1347]:
len(df2['주소'].unique())

4364

In [1348]:
df[df['주소'] == '경상남도 김해시 진영읍 하계로96번길 94-4 , 주유소 옆 주차장'].head()

,충전소명,지역,시군구,주소
2580,진영휴게소(부산방향),경상남도,김해시,"경상남도 김해시 진영읍 하계로96번길 94-4 , 주유소 옆 주차장"
2581,진영휴게소(부산방향),경상남도,김해시,"경상남도 김해시 진영읍 하계로96번길 94-4 , 주유소 옆 주차장"
7273,진영휴게소(부산방향),경상남도,김해시,"경상남도 김해시 진영읍 하계로96번길 94-4 , 주유소 옆 주차장"
7274,진영휴게소(부산방향),경상남도,김해시,"경상남도 김해시 진영읍 하계로96번길 94-4 , 주유소 옆 주차장"
7275,진영휴게소(부산방향),경상남도,김해시,"경상남도 김해시 진영읍 하계로96번길 94-4 , 주유소 옆 주차장"


In [1349]:
df2[df2['주소'] == '경상남도 김해시 진영읍 하계로96번길 94-4 , 주유소 옆 주차장']

,충전소명,지역,시군구,주소
858,진영휴게소(부산방향),경상남도,김해시,"경상남도 김해시 진영읍 하계로96번길 94-4 , 주유소 옆 주차장"


In [1350]:
adr[858]

'경상남도 김해시 진영읍 하계로96번길 94-4 , 주유소 옆 주차장'

### 특이주소 처리

In [1351]:
# 경상북도 경주시 천원2길 9 경북 경주시 교동 153-5
# 충청북도 단양군 영춘면 온달평강3로 9 온달평강3로 9-1
# 광주광역시 북구 첨단 벤처로 108번길 9

In [1352]:
count = 0
for i, one in enumerate(adr):
    if '경상북도 경주시 천원2길 9 경북 경주시 교동 153-5' in one:
        adr[i] = '경상북도 경주시 교동 천원2길 9'
        count += 1
count

1

In [1353]:
count = 0
for i, one in enumerate(adr):
    if '충청북도 단양군 영춘면 온달평강3로 9 온달평강3로 9-1' in one:
        adr[i] = '충청북도 단양군 영춘면 온달평강3로 9'
        count += 1
count

1

In [1354]:
count = 0
for i, one in enumerate(adr):
    if '광주광역시 북구 첨단 벤처로 108번길 9' in one:
        adr[i] = '광주광역시 북구 첨단벤처로108번길 9'
        count += 1
count

1

### 빈칸 2개 -> 1개

In [1355]:
count = 0
for i, one in enumerate(adr):
    if '  ' in one:
        adr[i] = one.replace('  ', ' ')
        count += 1
count

128

### 컴마(,) 제거

In [1356]:
count = 0
for i, one in enumerate(adr):
    if ',' in one:
        adr[i] = one.replace(',', '')
        count += 1
count

324

### 복수 주소 오류
- 한주소에 2개 주소 들어있는 오류
- 동일지역 : 56
- 다른지역 : 2

#### 동일지역

In [1357]:
np.where(adr == '전라남도 무안군 망운면 운해로 1436-10 전라남도 함평군 신광면 학동로 1398-9')

(array([2608], dtype=int64),)

In [1358]:
adr[2608]

'전라남도 무안군 망운면 운해로 1436-10 전라남도 함평군 신광면 학동로 1398-9'

In [1359]:
si = df2['시군구'].to_numpy()

In [1360]:
count = 0
for i, one in tqdm(enumerate(adr)):
    first = list(one.split())[0]
    one = one.replace(' ' + first, '$' + first)
    multi_adr = list(one.split('$'))
    if len(multi_adr) == 2:
        count += 1
        for j, two in enumerate(multi_adr):
            if si[i] in two:
                adr[i] = multi_adr[j]
count

4364it [00:00, 336667.57it/s]


56

In [1361]:
adr[2608]

'전라남도 무안군 망운면 운해로 1436-10'

#### 다른지역

In [1362]:
sep = ' 광주광역시| 전라남도| 전라북도| 제주특별자치도| 세종특별자치시| 서울특별시| 대전광역시| 경기도| 인천광역시| 경상남도| 강원도| 충청북도| 경상북도| 충청남도| 울산광역시| 대구광역시| 부산광역시'

In [1363]:
count = 0
for i, one in tqdm(enumerate(adr)):
    multi_local = list(re.split(sep, one))
    if len(multi_local) == 2:
        print(i, one)
        count += 1
count

4364it [00:00, 364613.11it/s]

2456 전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2
2791 충청북도 청주시 청원구 오창읍 구룡리 376-1 경상남도 거제시 양정동 665-1


2

In [1364]:
rep_dict = {" 광주광역시":"$광주광역시" ," 전라남도":"$전라남도"," 전라북도":"$전라북도"," 제주특별자치도":"$제주특별자치도"," 세종특별자치시":"$세종특별자치시"," 서울특별시":"$서울특별시"," 대전광역시":"$대전광역시"," 경기도":"$경기도"," 인천광역시":"$인천광역시"," 경상남도":"$경상남도"," 강원도":"$강원도"," 충청북도":"$충청북도"," 경상북도":"$경상북도"," 충청남도":"$충청남도"," 울산광역시":"$울산광역시"," 대구광역시":"$대구광역시"," 부산광역시":"$부산광역시"}

In [1365]:
np.where(adr == '전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2')

(array([2456], dtype=int64),)

In [1366]:
adr[2456]

'전라북도 순창군 풍산면 반월리 12-4 충청남도 공주시 탄천면 삼각리 523-2'

In [1367]:
adr[2791]

'충청북도 청주시 청원구 오창읍 구룡리 376-1 경상남도 거제시 양정동 665-1'

In [1368]:
count = 0
for i, one in tqdm(enumerate(adr)):
    for k, v in rep_dict.items():
        one = one.replace(k, v)
    multi_adr = list(one.split('$'))
    if len(multi_adr) == 2:
        count += 1
        for j, two in enumerate(multi_adr):
            if si[i] in two:
                adr[i] = multi_adr[j]
count

4364it [00:00, 137955.55it/s]


2

In [1369]:
adr[2456]

'전라북도 순창군 풍산면 반월리 12-4'

In [1370]:
adr[2791]

'충청북도 청주시 청원구 오창읍 구룡리 376-1'

### 도로명 주소 정리
- [길 숫자] 분리
- [로 숫자] 분리
- -문자 (숫자)(번)길 붙이기
    - [로 숫자(번)길] 붙이기
    - [앙 (숫자)길] 붙이기
    - [장 (숫자)길] 붙이기
    - [솜 (숫자)로] 붙이기
    - [길 (숫자)번길] 붙이기
    - [터 (숫자)길] 붙이기
    - [개 (숫자)길] 붙이기

#### [확인] 주소 분리
- 정상
    - 로
        - [한글]로
        - [한글][숫자]로
        - [한글][(숫자)]로[숫자]길
        - [한글][(숫자)]로[숫자][번|가|나|다|안]길
    - 길
        - [한글]길
        - [한글][숫자]길
        - [한글][숫자]길[숫자]번길
    - 동
        - [한글]동
        - [한글][숫자]동
    - 가
        - [한글]동[숫자]가
        - [한글]로[숫자]가


#### [분리] [한글][번길, 길, 동] [숫자]
- 주의
    - 분리 YES
        - 834, 경상남도 거창군 거창읍 심소정길39-36
        - 1201, 강원도 삼척시 수로부인길453
    - 분리 NO
        - 743, 서울특별시 영등포구 신길1동 116-15
        - 3673, 경상북도 포항시 남구 구룡포읍 구룡포길117번길 28-8

In [1371]:
np.where(adr == '전라북도 전주시 완산구 신촌3길24번지')

(array([15], dtype=int64),)

In [1372]:
adr[15]

'전라북도 전주시 완산구 신촌3길24번지'

In [1373]:
split1 = re.compile('([가-힣]+[0-9]*[번동안가나길])([0-9]+)')

In [1374]:
print(adr[15])
print(split1.search(adr[15]))
split1.sub('\g<1> \g<2>', adr[15])

전라북도 전주시 완산구 신촌3길24번지
<re.Match object; span=(13, 19), match='신촌3길24'>


'전라북도 전주시 완산구 신촌3길 24번지'

In [1375]:
print(adr[743])
print(split1.search(adr[743]))

서울특별시 영등포구 신길1동 116-15
<re.Match object; span=(11, 14), match='신길1'>


In [1376]:
print(adr[3673])
print(split1.search(adr[3673]))

경상북도 포항시 남구 구룡포읍 구룡포길117번길 28-8
<re.Match object; span=(17, 24), match='구룡포길117'>


In [1377]:
count = 0
for i, one in enumerate(adr):
    if split1.search(one):
        print(i, one)
        count += 1

15 전라북도 전주시 완산구 신촌3길24번지
16 제주특별자치도 제주시 연동2334-4
120 충청북도 제천시 의림대로42길34
163 경상남도 창원시 성산구 사파동64-2
389 충청북도 단양군 적성면 기동2길 32-37 (기동리)
439 전라북도 전주시 완산구 효자동3가 1712-10
443 전라북도 전주시 완산구 효자동2가 1234-3 5층
547 서울특별시 송파구 오금로68길8
559 제주특별자치도 제주시 한림읍 금악서길133
605 전라북도 전주시 덕진구 인후동1가 807-6
645 경상북도 구미시 인동14길 14
649 대구광역시 북구 노원동2가 214
687 경상남도 창원시 마산합포구 신마산시장길 33 (창포동3가)
717 서울특별시 송파구 성내천로33가길20
743 서울특별시 영등포구 신길1동 116-15
834 경상남도 거창군 거창읍 심소정길39-36
854 경상남도 창원시 진해구 용재로 46  웅동2동 주민센터 주차장 내
887 경기도 파주시 탄현면 헤이리마을길82-105
982 서울특별시 종로구 인사동5길 29 지상주차장
1000 제주특별자치도 제주시 조천읍 선흘서5길7
1056 경상남도 창원시 장군동4가 3-6
1062 부산광역시 중구 중앙대로2 (중앙동7가)
1135 충청북도 청주시 서원구 현도면 선동1길 9
1139 충청남도 당진시 무수동2길 25-21
1201 강원도 삼척시 수로부인길453
1206 대전광역시 동구 용운동423-22
1332 충청북도 청주시 흥덕구 직지대로620번길70
1342 충청북도 괴산군 청안면 청안읍내로2길10
1368 울산광역시 울주군 언양읍 언동3길 20-48
1377 대전광역시 동구 홍도동92-3
1426 강원도 삼척시 당저동1길 17-9
1479 경기도 양평군 개군면 하자포길33번길 8
1568 경상북도 구미시 오태동659-13
1570 경상북도 포항시 북구 흥해읍 초곡길36번길 30
1647 강원도 태백시 소도동326
1675 충청북도 괴산군 불정면 목도로2길11
1748 인천광역시 중구 북성동1가 9

In [1378]:
count

109

In [1379]:
# 16 제주특별자치도 제주시 연동2334-4
# 163 경상남도 창원시 성산구 사파동64-2
# 559 제주특별자치도 제주시 한림읍 금악서길133
# 834 경상남도 거창군 거창읍 심소정길39-36
# 887 경기도 파주시 탄현면 헤이리마을길82-105
# 1062 부산광역시 중구 중앙대로2 (중앙동7가)
# 1201 강원도 삼척시 수로부인길453
# 1206 대전광역시 동구 용운동423-22
# 1377 대전광역시 동구 홍도동92-3
# 1568 경상북도 구미시 오태동659-13
# 1647 강원도 태백시 소도동326
# 1811 경상남도 창원시 진해구 명동580 좌측 주차장
# 1819 울산광역시 중구 단장골길1
# 1919 강원도 삼척시 근덕면 상맹방길30-80
# 1954 경상남도 함양군 안의면 농월정길9-13
# 2058 울산광역시 울주군 웅촌면 새초천길12
# 2396 울산광역시 남구 선암동584-8번지
# 2586 경상북도 의성군 비안면 강변길59
# 2694 경상북도 포항시 장기면 양포항길12
# 2780 제주특별자치도 제주시 용두암길15
# 2851 강원도 태백시 보드미길8(장성동)
# 2976 울산광역시 울주군 삼동면 보삼길550
# 2990 강원도 강릉시 성산면 삼포암길133
# 3030 경상북도 봉화군 봉성면 부랭이길88
# 3043 전라남도 보성군 용문길18
# 3059 전라남도 보성군 미력면 가평길44
# 3061 경상북도 영양군 영양읍 군청길37
# 3088 경상남도 함양군 함양읍 삼휴길60-30
# 3174 경상북도 청송군 현서면 달정길9
# 3177 경상북도 청송군 주왕산면 이전앞길27
# 3189 경상북도 영양군 영양읍 군민회관길46
# 3211 충청북도 영동군 용화면 휴양림길60
# 3272 강원도 춘천시 칠전동길28
# 3273 전라북도 군산시 창길21
# 3399 전라남도 보성군 회천면 우암길21
# 3401 충청남도 금산군 금성면 의총길25
# 3413 경상북도 구미시 광평길55
# 3550 경상북도 안동시 학가산온천길42

In [1380]:
# 15 전라북도 전주시 완산구 신촌3길24번지
# 120 충청북도 제천시 의림대로42길34
# 547 서울특별시 송파구 오금로68길8
# 717 서울특별시 송파구 성내천로33가길20
# 1000 제주특별자치도 제주시 조천읍 선흘서5길7
# 1332 충청북도 청주시 흥덕구 직지대로620번길70
# 1342 충청북도 괴산군 청안면 청안읍내로2길10
# 1675 충청북도 괴산군 불정면 목도로2길11
# 1854 강원도 철원군 동송읍 이평2로17번길32
# 1951 충청북도 괴산군 장연면 미선로오가5길7
# 2162 서울특별시 강서구 화곡로65길62
# 2747 부산광역시 강서구 공항진입로42번길20
# 2970 경상남도 합천군 합천읍 교동1길33
# 2979 충청북도 괴산군 칠성면 명태재로외사4길1
# 2999 경상남도 창녕군 길곡면 증산2길19
# 3008 경상북도 칠곡군 동명면 남원로5길77
# 3017 충청남도 부여군 양화면 입포로53번길6
# 3042 인천광역시 중구 신포로27번길80
# 3069 경상남도 창원시 마산합포구 현동7길27
# 3078 대구광역시 동구 첨단로8길32
# 3079 충청남도 논산시 산양1길2
# 3152 충청북도 영동군 황간면 원촌동1길54-8
# 3165 경상북도 봉화군 춘양면 의양로5길19
# 3192 경상북도 청송군 진보면 이촌1길23-1
# 3197 충청북도 제천시 내토로16안길8-1
# 3224 경기도 시흥시 시흥대로268번길8
# 3324 전라북도 군산시 미장8길7
# 3376 경상북도 영주시 중앙로63번길41-11
# 3387 전라남도 목포시 남악로162번길25
# 3396 울산광역시 울주군 청량읍 처용산업4길51
# 3419 경상남도 창원시 의창구 태복산로15번길8
# 3424 경상남도 진주시 가좌길29번길63
# 3438 경상북도 칠곡군 왜관읍 평장3길47

In [1381]:
# 3025 경상남도 김해시 삼안로 112번길20-61
# 3073 대전광역시 유성구 송림로 19번길35
# 3101 대전광역시 유성구 노은동로 87번길89
# 3742 경상남도 창원시 의창구 사림로 99번길63

In [1382]:
idx11 = [16, 163, 559, 834, 887, 1062, 1201, 1206, 1377, 1568, 1647, 1811, 1819, 1919, 
        1954, 2058, 2396, 2586, 2694, 2780, 2851, 2976, 2990, 3030, 3043,
        3059, 3061, 3088, 3174, 3177, 3189, 3211, 3272, 3273, 3399, 3401, 3413, 3550]
len(idx11)

38

In [1383]:
idx12 = [15, 120, 547, 717, 1000, 1332, 1342, 1675, 1854, 1951, 
         2162, 2747, 2970, 2979, 2999, 3008, 3017, 3042, 3069, 3078,
         3079, 3152, 3165, 3192, 3197, 3224, 3324, 3376, 3387,
         3396, 3419, 3424, 3438]
len(idx12)

33

In [1384]:
idx13 = [3025, 3073, 3101, 3742]

In [1385]:
split11 = re.compile('([가-힣]+[0-9]*[번동안가나길로])([0-9]+)')

In [1386]:
print(adr[16])
print(split11.search(adr[16]))
print(split11.sub('\g<1> \g<2>', adr[16], 1) )

제주특별자치도 제주시 연동2334-4
<re.Match object; span=(12, 18), match='연동2334'>
제주특별자치도 제주시 연동 2334-4


In [1387]:
print(adr[1062])
print(split11.search(adr[1062]))
print(split11.sub('\g<1> \g<2>', adr[1062], 1) )

부산광역시 중구 중앙대로2 (중앙동7가)
<re.Match object; span=(9, 14), match='중앙대로2'>
부산광역시 중구 중앙대로 2 (중앙동7가)


In [1388]:
print(adr[2396])
print(split11.search(adr[2396]))
print(split11.sub('\g<1> \g<2>', adr[2396], 1) )

울산광역시 남구 선암동584-8번지
<re.Match object; span=(9, 15), match='선암동584'>
울산광역시 남구 선암동 584-8번지


In [1389]:
print(adr[2851])
print(split11.search(adr[2851]))
print(split11.sub('\g<1> \g<2>', adr[2851], 1) )

강원도 태백시 보드미길8(장성동)
<re.Match object; span=(8, 13), match='보드미길8'>
강원도 태백시 보드미길 8(장성동)


In [1390]:
print(adr[3399])
print(split11.search(adr[3399]))
print(split11.sub('\g<1> \g<2>', adr[3399], 1) )

전라남도 보성군 회천면 우암길21
<re.Match object; span=(13, 18), match='우암길21'>
전라남도 보성군 회천면 우암길 21


In [1391]:
adr[[16, 1062, 2396, 2851, 3399]]

array(['제주특별자치도 제주시 연동2334-4', '부산광역시 중구 중앙대로2 (중앙동7가)',
       '울산광역시 남구 선암동584-8번지', '강원도 태백시 보드미길8(장성동)', '전라남도 보성군 회천면 우암길21'],
      dtype=object)

In [1392]:
len(idx11)

38

In [1393]:
for i in tqdm(idx11):
    temp = adr[i]
    adr[i] = split11.sub('\g<1> \g<2>', temp, 1)

100%|██████████| 38/38 [00:00<00:00, 38194.00it/s]


In [1394]:
adr[[16, 1062, 2396, 2851, 3399]]

array(['제주특별자치도 제주시 연동 2334-4', '부산광역시 중구 중앙대로 2 (중앙동7가)',
       '울산광역시 남구 선암동 584-8번지', '강원도 태백시 보드미길 8(장성동)',
       '전라남도 보성군 회천면 우암길 21'], dtype=object)

In [1395]:
split12 = re.compile('([가-힣]+[0-9]*[번길])([0-9]+)')

In [1396]:
print(adr[15])
print(split12.search(adr[15]))
print(split12.sub('\g<1> \g<2>', adr[15], 1) )

전라북도 전주시 완산구 신촌3길24번지
<re.Match object; span=(13, 19), match='신촌3길24'>
전라북도 전주시 완산구 신촌3길 24번지


In [1397]:
print(adr[1854])
print(split12.search(adr[1854]))
print(split12.sub('\g<1> \g<2>', adr[1854], 1) )

강원도 철원군 동송읍 이평2로17번길32
<re.Match object; span=(18, 22), match='번길32'>
강원도 철원군 동송읍 이평2로17번길 32


In [1398]:
print(adr[1000])
print(split12.search(adr[1000]))
print(split12.sub('\g<1> \g<2>', adr[1000], 1) )

제주특별자치도 제주시 조천읍 선흘서5길7
<re.Match object; span=(16, 22), match='선흘서5길7'>
제주특별자치도 제주시 조천읍 선흘서5길 7


In [1399]:
print(adr[3152])
print(split12.search(adr[3152]))
print(split12.sub('\g<1> \g<2>', adr[3152], 1) )

충청북도 영동군 황간면 원촌동1길54-8
<re.Match object; span=(13, 20), match='원촌동1길54'>
충청북도 영동군 황간면 원촌동1길 54-8


In [1400]:
adr[[15, 1854, 1000, 3152]]

array(['전라북도 전주시 완산구 신촌3길24번지', '강원도 철원군 동송읍 이평2로17번길32',
       '제주특별자치도 제주시 조천읍 선흘서5길7', '충청북도 영동군 황간면 원촌동1길54-8'], dtype=object)

In [1401]:
len(idx12)

33

In [1402]:
for i in tqdm(idx12):
    temp = adr[i]
    adr[i] = split12.sub('\g<1> \g<2>', temp, 1)

100%|██████████| 33/33 [00:00<00:00, 33152.58it/s]


In [1403]:
adr[[15, 1854, 1000, 3152]]

array(['전라북도 전주시 완산구 신촌3길 24번지', '강원도 철원군 동송읍 이평2로17번길 32',
       '제주특별자치도 제주시 조천읍 선흘서5길 7', '충청북도 영동군 황간면 원촌동1길 54-8'], dtype=object)

In [1404]:
split13 = re.compile('([가-힣]+로) ([0-9]+번길)([0-9]+)')

In [1405]:
print(adr[3025])
print(split13.search(adr[3025]))
print(split13.sub('\g<1>\g<2> \g<3>', adr[3025], 1) )

경상남도 김해시 삼안로 112번길20-61
<re.Match object; span=(9, 20), match='삼안로 112번길20'>
경상남도 김해시 삼안로112번길 20-61


In [1406]:
print(adr[3101])
print(split13.search(adr[3101]))
print(split13.sub('\g<1>\g<2> \g<3>', adr[3101], 1) )

대전광역시 유성구 노은동로 87번길89
<re.Match object; span=(10, 21), match='노은동로 87번길89'>
대전광역시 유성구 노은동로87번길 89


In [1407]:
for i in tqdm(idx13):
    temp = adr[i]
    adr[i] = split13.sub('\g<1>\g<2> \g<3>', temp, 1)

100%|██████████| 4/4 [00:00<00:00, 4005.06it/s]


In [1408]:
adr[[3025, 3101]]

array(['경상남도 김해시 삼안로112번길 20-61', '대전광역시 유성구 노은동로87번길 89'], dtype=object)

#### [분리] -길-로-번길 [숫자]-[숫자]

In [1409]:
split2 = re.compile('([가-힣]+[로가나번길동리])([0-9]+[-][0-9]+)')

In [1410]:
count = 0
idx2 = []
for i, one in enumerate(adr):
    if split2.search(one):
        idx2.append(i)
        print(i, one)
        count += 1
count

19 제주특별자치도 제주시 애월읍 애월해안로400-9
132 경기도 화성시 노작로226-26
771 강원도 철원군 서면 와수리1160-3
1058 경상남도 합천군 쌍백면 쌍백중앙로162-8
1336 충청북도 청주시 청원구 오창읍 두릉유리로1137-1
1519 서울특별시 송파구 성내천로299-7
1618 대구광역시 달성군 가창면 가창로607-2
1648 강원도 속초시 설악산로470-7
1709 경상남도 거제시 일운면 와현리431-2
1905 울산광역시 울주군 범서읍 구영리216-3
1978 경기도 파주시 적성면 설마리48-3
2104 경상남도 산청군 시천면 중산리501-4
2443 충청남도 부여군 홍산면 북촌리188-7
2447 강원도 삼척시 원덕읍 삼척로445-53
2961 경상남도 김해시 삼계로104-32
2966 경상남도 통영시 산양읍 산양일주로1683-25
2987 경상북도 청송군 주왕산면 주왕산로698-35
3005 경상북도 영주시 장수면 용주로88-42
3072 경상북도 청송군 청송읍 금월로244-30
3097 경상남도 합천군 묘산면 묘산로173-1
3128 경상북도 영양군 입암면 영양로883-5
3210 경상북도 청송군 부남면 청송로3478-96
3262 전라북도 고창군 해리면 하련리189-5
3286 경상남도 의령군 의령읍 충익로86-2
3433 경상북도 울진군 대흥신림로1046-280
3551 경상남도 의령군 지정면 봉곡리511-2
3723 경상북도 구미시 선산읍 선산중앙로83-2


27

In [1411]:
print(adr[771])
print(split2.search(adr[771]))
print(split2.sub('\g<1> \g<2>', adr[771], 1) )

강원도 철원군 서면 와수리1160-3
<re.Match object; span=(11, 20), match='와수리1160-3'>
강원도 철원군 서면 와수리 1160-3


In [1412]:
for i in tqdm(idx2):
    temp = adr[i]
    adr[i] = split2.sub('\g<1> \g<2>', temp, 1)

100%|██████████| 27/27 [00:00<?, ?it/s]


In [1413]:
adr[[771, 3723]]

array(['강원도 철원군 서면 와수리 1160-3', '경상북도 구미시 선산읍 선산중앙로 83-2'], dtype=object)

#### [확인] [한글][(숫자)]로[숫자][번|가|나|안]길

In [1414]:
split3 = re.compile('([가-힣]+[0-9]*[로][0-9]+[가-힣][번가나안길])([0-9]+)')

In [1415]:
adr[3224]

'경기도 시흥시 시흥대로268번길 8'

In [1416]:
s = '경기도 시흥시 시흥대로268번길8'
split3.search(s)

<re.Match object; span=(8, 18), match='시흥대로268번길8'>

In [1417]:
count = 0
idx3 = []
for i, one in enumerate(adr):
    if split3.search(one):
        print(i, one)
        idx3.append(i)
        count += 1

In [1418]:
count

0

#### [확인] [한글]로

In [1419]:
adr[2993]

'경상남도 창녕군 유어면 우포1대로583'

In [1420]:
split4 = re.compile('([가-힣]+로)([0-9]+)')

In [1421]:
print(adr[2993])
print(split4.search(adr[2993]))

경상남도 창녕군 유어면 우포1대로583
<re.Match object; span=(16, 21), match='대로583'>


In [1422]:
for i, one in tqdm(enumerate(adr)):
    if split4.search(one):
        print(i, one)

4364it [00:00, 273654.71it/s]

1 광주광역시 북구 첨단벤처로108번길 9
4 전라남도 구례군 산동면 노고단로1068
18 제주특별자치도 서귀포시 성산읍 해맞이해안로2660
35 대전광역시 유성구 반석로12번길 18-4
43 서울특별시 마포구 마포대로1길 9
44 서울특별시 성동구 광나루로4가길 23
65 충청북도 충주시 으뜸로21
79 제주특별자치도 제주시 중앙로25길 17
85 경기도 이천시 마장면 중부고속도로82
88 경기도 용인시 처인구 주북로94번길 30-1 (고림동)
100 경기도 시흥시 서울대학로278번길 67 지상4층 주차장 A4
101 경상남도 양산시 백호로23 축구장 옆
103 서울특별시 영등포구 의사당대로1 국회의원회관 주차장 우측
116 경기도 가평군 설악면 미사리로540번길 51
120 충청북도 제천시 의림대로42길 34
131 경기도 화성시 동탄공원로2길 32
138 제주특별자치도 서귀포시 소보리당로164번길 62  주차장 좌측
177 대구광역시 동구 아양로37길 92
178 대구광역시 북구 복현로8길 16-5
215 경기도 이천시 마장면 중부대로609번길 244-35 (목리)
216 경기도 이천시 신둔면 서이천로853번길 346-129 (용면리)
232 전라남도 곡성군 곡성읍 학교로8
296 경상남도 김해시 장유로116번길 44-82 (부곡동)
316 경기도 동두천시 평화로2910번길 30
322 경기도 양평군 청운면 용두로164번길 9-5
324 경기도 이천시 마장면 덕이로154번길 287-76 휴게소 옥외 장애인 주차장(휴게소 정면 우측 흡연장 앞)
326 경기도 군포시 군포로609번안길 12
369 서울특별시 강동구 진황도로29길 34 지하1층 26
374 서울특별시 관악구 난곡로34나길 14 2층
375 서울특별시 관악구 낙성대로4가길 5
379 서울특별시 노원구 동일로204가길 12 5층 출구앞
402 서울특별시 영등포구 의사당대로1 국회도서관 앞
409 강원도 삼척시 청석로3길 16-41
421 제주특별자치도 서귀포시 표선면 성읍정의현로22번길 7-1  

In [1423]:
# 4 전라남도 구례군 산동면 노고단로1068
# 18 제주특별자치도 서귀포시 성산읍 해맞이해안로2660
# 65 충청북도 충주시 으뜸로21
# 85 경기도 이천시 마장면 중부고속도로82
# 101 경상남도 양산시 백호로23 축구장 옆
# 103 서울특별시 영등포구 의사당대로1 국회의원회관 주차장 우측
# 232 전라남도 곡성군 곡성읍 학교로8
# 402 서울특별시 영등포구 의사당대로1 국회도서관 앞
# 465 경상북도 성주군 성주읍 성주순환로251
# 543 서울특별시 강남구 남부순환로3104
# 565 경상북도 청송군 파천면 청송로5544
# 665 경상북도 포항시 북구 해안로82
# 693 서울특별시 강남구 테헤란로606
# 770 강원도 철원군 갈말읍 삼부연로38
# 905 서울특별시 영등포구 양산로111  왼쪽 끝편 주차장
# 995 제주특별자치도 서귀포시 상효로39 (상효동 754-1)
# 1057 경상남도 창원시 의창구 창원대로314
# 1192 경상북도 의성군 안계면 서부로1783
# 1205 강원도 원주시 입춘로65
# 1324 전라남도 영광군 백수읍 해안로918
# 1338 충청북도 청주시 서원구 수곡로61
# 1341 충청북도 괴산군 연풍면 향교로71
# 1505 경상남도 양산시 어곡터널로40
# 1506 경상남도 양산시 물금읍 황산로501
# 1507 경상남도 의령군 의령읍 백산로4
# 1522 서울특별시 양천구 목동동로81
# 1526 강원도 평창군 진부면 경강로3562
# 1566 경상북도 상주시 화남면 문장로758
# 1569 경상북도 포항시 남구 장기면 동해안로2987
# 1581 경기도 의정부시 동일로30
# 1606 경상북도 영천시 천문로594
# 1619 경상북도 경산시 진량읍 공단로299
# 1620 경상북도 상주시 함창로581번지
# 1639 대구광역시 달성군 구지면 구지서로1
# 1765 경상북도 영천시 한방로53번지
# 1766 강원도 삼척시 새천년도로673
# 1768 대전광역시 대덕구 신탄진로556
# 1777 경기도 양주시 은현면 은남로177
# 1783 전라남도 영광군 천년로1329
# 1844 경상북도 영천시 한방로77
# 1850 전라북도 남원시 운봉읍 황산로1026
# 1871 경기도 시흥시 시흥대로869
# 1914 울산광역시 북구 산업로1103
# 1923 경기도 시흥시 동서로425
# 1943 충청남도 논산시 채운면 채운로41
# 1983 서울특별시 노원구 노원로75
# 2004 경상북도 상주시 영남제일로1950
# 2056 전라남도 영광군 영광읍 영광로1330
# 2085 인천광역시 서구 중봉대로211
# 2089 충청북도 진천군 진광로487
# 2190 광주광역시 광산구 풍영로179
# 2328 경기도 군포시 산본로267
# 2627 경상북도 경주시 산내면 문복로1822
# 2704 경상남도 진주시 금산면 송백로46
# 2957 충청남도 부여군 충화면 팔충로770
# 2958 경상북도 구미시 선산읍 김선로963
# 2959 경상남도 합천군 합천읍 대야로991
# 2960 경상남도 창녕군 창녕읍 탐하로201
# 2962 충청남도 보령시 대천로113
# 2965 충청북도 음성군 맹동면 원중로1339
# 2969 경상북도 구미시 도개면 도안로103
# 2971 경상남도 의령군 유곡면 청정로1679
# 2973 경상남도 창녕군 성산면 성산로237
# 2974 경상북도 구미시 박정희로107
# 2975 경상남도 합천군 적중면 적중로98
# 2978 충청북도 영동군 용산면 영동산단로27
# 2980 경상북도 문경시 중앙로50
# 2984 충청북도 영동군 매곡면 민주지산로3595
# 2988 전라남도 영암군 삼호읍 공항로21
# 2993 경상남도 창녕군 유어면 우포1대로583
# 2995 경상남도 합천군 용주면 황계폭포로1154
# 2996 경상북도 청송군 주왕산면 부동로1008
# 3010 경상북도 청송군 파천면 안파로1244
# 3011 전라남도 장흥군 용산면 장흥대로2667
# 3013 경상남도 거창군 신원면 신차로2924
# 3016 경상북도 성주군 성주읍 주산로193
# 3021 경상북도 구미시 경부고속도로172
# 3032 경상남도 창원시 의창구 소계로135
# 3033 경상남도 창원시 진해구 웅천중로59
# 3034 경상북도 청송군 진보면 청송로6359
# 3040 경상북도 영양군 영양읍 중앙로55
# 3041 울산광역시 울주군 청량면 군청로1
# 3044 충청남도 청양군 비봉면 충절로1994
# 3049 충청남도 서산시 문화로112
# 3053 경상북도 봉화군 상운면 예봉로1281
# 3056 충청남도 부여군 은산면 충의로694
# 3060 경기도 파주시 시청로66
# 3066 경상남도 통영시 통영해안로2
# 3067 충청남도 부여군 세도면 세도중앙로8
# 3070 경상북도 영주시 풍기읍 소백로2156
# 3074 인천광역시 남동구 남동서로209
# 3080 경상북도 청송군 청송읍 주왕산로177
# 3083 경상남도 합천군 초계면 동부로1240
# 3084 충청남도 보령시 천북면 홍보로462
# 3085 경상남도 의령군 부림면 입산로178
# 3090 경상북도 청송군 청송읍 군청로25
# 3092 경상남도 합천군 대양면 대한로5
# 3093 대전광역시 유성구 신성로55
# 3096 경상북도 구미시 형곡로180
# 3098 경상남도 의령군 낙서면 낙동강로859
# 3100 경상북도 안동시 관광단지로316
# 3132 충청남도 예산군 삽교읍 예학로65
# 3169 경상북도 청송군 안덕면 방호정로50
# 3186 경상북도 청송군 안덕면 안덕로82
# 3187 광주광역시 북구 하서로111
# 3194 경상북도 청송군 청송읍 중앙로184
# 3198 충청북도 제천시 세명로21
# 3199 대전광역시 유성구 유성대로1719
# 3216 경기도 화성시 송산면 평택시흥고속도로25
# 3240 전라북도 정읍시 내장호반로266
# 3252 전라북도 익산시 왕궁면 궁성로666
# 3280 강원도 정선군 사북읍 사북중앙로65
# 3297 경기도 구리시 장자대로141
# 3363 경상북도 영주시 대학로352
# 3364 경기도 화성시 팔탄면 서해안고속도로302
# 3365 경기도 용인시 수지구 경부고속도로400
# 3372 경상남도 밀양시 산내면 얼음골로177
# 3373 경상북도 의성군 사곡면 의성사곡로966
# 3378 경상북도 의성군 안사면 안신로645
# 3409 전라북도 남원시 이백로13
# 3422 경상북도 영주시 영주로204
# 3427 경상북도 문경시 중앙로280
# 3430 경상남도 밀양시 산내면 얼음골로334
# 3431 경상북도 구미시 송정대로89
# 3432 경상북도 구미시 해마루공원로24
# 3435 경상북도 울진군 온정면 백암온천로1308
# 3436 경상북도 예천군 효자면 은풍로1154
# 3439 경상북도 울진군 온정면 백암온천로1185
# 3470 광주광역시 서구 천변우하로391
# 3518 경상남도 창원시 마산합포구 진동면 신서촌로15
# 3669 충청북도 청주시 서원구 흥덕로48
# 3694 충청남도 아산시 실옥로234
# 3700 광주광역시 북구 추암로249
# 3726 경상남도 산청군 산청읍 물안실로8
# 3886 울산광역시 북구 천곡남로86
# 3887 울산광역시 남구 갈밭로49

In [1424]:
idx4 = [4, 18, 65, 85, 101, 103, 232, 402, 465, 543, 565, 665, 693, 
        770, 905, 995, 1057, 1192, 1205, 1324, 1338, 1341, 1505, 1506, 
        1507, 1522, 1526, 1566, 1569, 1581, 1606, 1619, 1620, 1639, 1765, 
        1766, 1768, 1777, 1783, 1844, 1850, 1871, 1914, 1923, 1943, 1983, 
        2004, 2056, 2085, 2089, 2190, 2328, 2627, 2704, 2957, 2958, 2959, 
        2960, 2962, 2965, 2969, 2971, 2973, 2974, 2975, 2978, 2980, 2984, 
        2988, 2993, 2995, 2996, 3010, 3011, 3013, 3016, 3021, 3032, 3033, 
        3034, 3040, 3041, 3044, 3049, 3053, 3056, 3060, 3066, 3067, 3070, 
        3074, 3080, 3083, 3084, 3085, 3090, 3092, 3093, 3096, 3098, 3100, 
        3132, 3169, 3186, 3187, 3194, 3198, 3199, 3216, 3240, 3252, 3280, 
        3297, 3363, 3364, 3365, 3372, 3373, 3378, 3409, 3422, 3427, 3430, 
        3431, 3432, 3435, 3436, 3439, 3470, 3518, 3669, 3694, 3700, 3726,
        3886, 3887]
len(idx4)

136

In [1425]:
print(adr[995])
print(split4.search(adr[995]))
print(split4.sub('\g<1> \g<2>', adr[995], 1))

제주특별자치도 서귀포시 상효로39 (상효동 754-1)
<re.Match object; span=(13, 18), match='상효로39'>
제주특별자치도 서귀포시 상효로 39 (상효동 754-1)


In [1426]:
print(adr[3409])
print(split4.search(adr[3409]))
print(split4.sub('\g<1> \g<2>', adr[3409], 1))

전라북도 남원시 이백로13
<re.Match object; span=(9, 14), match='이백로13'>
전라북도 남원시 이백로 13


In [1427]:
print(adr[402])
print(split4.search(adr[402]))
print(split4.sub('\g<1> \g<2>', adr[402], 1))

서울특별시 영등포구 의사당대로1 국회도서관 앞
<re.Match object; span=(11, 17), match='의사당대로1'>
서울특별시 영등포구 의사당대로 1 국회도서관 앞


In [1428]:
for i in tqdm(idx4):
    temp = adr[i]
    adr[i] = split4.sub('\g<1> \g<2>', temp, 1)

100%|██████████| 136/136 [00:00<00:00, 137055.58it/s]


In [1429]:
adr[[402, 995, 3409]]

array(['서울특별시 영등포구 의사당대로 1 국회도서관 앞', '제주특별자치도 서귀포시 상효로 39 (상효동 754-1)',
       '전라북도 남원시 이백로 13'], dtype=object)

In [1430]:
adr[2993]

'경상남도 창녕군 유어면 우포1대로 583'

#### [확인] [한글]길

In [1431]:
split5 = re.compile('([가-힣]+[0-9]*길)([0-9]+)')

In [1432]:
count = 0
for i, one in tqdm(enumerate(adr)):
    if split5.search(one):
        count += 1
        print(i, one)

4364it [00:00, 437589.77it/s]

743 서울특별시 영등포구 신길1동 116-15
1479 경기도 양평군 개군면 하자포길33번길 8
1570 경상북도 포항시 북구 흥해읍 초곡길36번길 30
3424 경상남도 진주시 가좌길 29번길63
3673 경상북도 포항시 남구 구룡포읍 구룡포길117번길 28-8


In [1433]:
print(adr[3424])
print(split5.search(adr[3424]))

경상남도 진주시 가좌길 29번길63
<re.Match object; span=(15, 19), match='번길63'>


In [1434]:
print(adr[3424])
if adr[3424] == '경상남도 진주시 가좌길 29번길63':
    adr[3424] = '경상남도 진주시 가좌길29번길 63'
print(adr[3424])

경상남도 진주시 가좌길 29번길63
경상남도 진주시 가좌길29번길 63


#### [확인] [한글]동

In [1435]:
split6 = re.compile('([가-힣]+[0-9]*동)([0-9]+)')

In [1436]:
count = 0
for i, one in tqdm(enumerate(adr)):
    if split6.search(one):
        count += 1
        print(i, one)

4364it [00:00, 336686.15it/s]

389 충청북도 단양군 적성면 기동2길 32-37 (기동리)
439 전라북도 전주시 완산구 효자동3가 1712-10
443 전라북도 전주시 완산구 효자동2가 1234-3 5층
605 전라북도 전주시 덕진구 인후동1가 807-6
645 경상북도 구미시 인동14길 14
649 대구광역시 북구 노원동2가 214
687 경상남도 창원시 마산합포구 신마산시장길 33 (창포동3가)
854 경상남도 창원시 진해구 용재로 46  웅동2동 주민센터 주차장 내
982 서울특별시 종로구 인사동5길 29 지상주차장
1056 경상남도 창원시 장군동4가 3-6
1062 부산광역시 중구 중앙대로 2 (중앙동7가)
1135 충청북도 청주시 서원구 현도면 선동1길 9
1139 충청남도 당진시 무수동2길 25-21
1368 울산광역시 울주군 언양읍 언동3길 20-48
1426 강원도 삼척시 당저동1길 17-9
1748 인천광역시 중구 북성동1가 97-32
2421 전라북도 전주시 덕진구 송천동2가 1257
2423 전라북도 전주시 덕진구 송천동2가 1316
2616 경상남도 창원시 마산회원구 석전동4길 52
2854 인천광역시 중구 송월동2가 4
2862 전라북도 전주시 덕진구 가련산로 26-3 (송천동1가)
2970 경상남도 합천군 합천읍 교동1길 33
3069 경상남도 창원시 마산합포구 현동7길 27
3152 충청북도 영동군 황간면 원촌동1길 54-8
3285 전라북도 익산시 인화동2가 7
3453 전라남도 여수시 삼동3길 13
3891 경상남도 고성군 거류면 당동3길 5
3924 전라남도 여수시 학동2길 7
3987 전라남도 영암군 신북면 황금동1길 5-1
4050 충청북도 청주시 서원구 현도면 선동2길 232-9
4181 전라남도 여수시 관문동1길 39
4221 충청북도 단양군 적성면 기동2길 32-37


In [1437]:
# 이상없음

#### [확인] 주소 분리
- 정상
    - 로
        - [한글]로
        - [한글][숫자]로
        - [한글][(숫자)]로[숫자]길
        - [한글][(숫자)]로[숫자][번|가|나|안]길
    - 길
        - [한글]길
        - [한글][숫자]길
        - [한글][숫자]길[숫자]번길
    - 동
        - [한글]동
        - [한글][숫자]동
    - 가
        - [한글]동[숫자]가
        - [한글]로[숫자]가


##### -로 체크

In [1438]:
check1 = re.compile('([가-힣]+[0-9]*로) ([0-9]+[번가나안길])')
for i, one in enumerate(adr):
    if check1.search(one):
        print(i, one)

27 대전광역시 대덕구 석봉로 38번길 55 대덕구보건소
94 광주광역시 서구 경열로 17번길 5
145 제주특별자치도 서귀포시 성산읍 일주동로 4120번길 7 동부소방서
189 충청남도 논산시 연무로 166번길 64 2주차장
196 제주특별자치도 제주시 선덕로 8길 12 설문대여성문화센터
236 대전광역시 중구 동서대로 1322번길 81
311 경기도 가평군 설악면 미사리로 540번길 51 가평휴게소  SK 가평주유소(춘천방향) 옆
362 광주광역시 광산구 광산로 68번길 13
365 제주특별자치도 서귀포시 동홍중앙로 7번길 19 자기주도학습지원센터
406 충청남도 홍성군 홍북읍 홍북로 453번길 50
433 대구광역시 달성군 구지면 창리로 11길 38
444 제주특별자치도 서귀포시 남성중로 153번길 15
446 제주특별자치도 서귀포시 효돈로 152번길 7
480 전라남도 여수시 학동 168(시청로 1번지)
535 서울특별시 영등포구 여의대방로 53길 22
537 서울특별시 강북구 도봉로 69길 29
578 서울특별시 서초구 남부순환로 340길 58
646 경상북도 구미시 봉곡로 15길 54
675 부산광역시 중구 대청로 116번길 3
724 인천광역시 중구 공항동로 296번길 69
739 강원도 철원군 철원읍 금학로 330번길 12
827 전라남도 화순군 능주면 학포로 2275번지
867 경기도 고양시 덕양구 화중로 104번길 13 입구 좌측
888 경기도 포천시 소홀읍 호국로 56번지
997 제주특별자치도 서귀포시 대정읍 추사로 36번길 1
1049 서울특별시 중구 소공로 51번지
1091 경기도 고양시 일산동구 중앙로 1275번길 64
1094 서울특별시 성동구 고산자로 26길 6
1111 경기도 이천시 마장면 덕이로 154번길 287-76
1116 경기도 성남시 분당구 야탑로 111번길 12-14
1117 경기도 성남시 분당구 성남대로 43번길 11
1214 충청북도 제천시 용두대로 11길 10
1234 서울특별시 양천구 남부순환로 88길 5-7
1

In [1439]:
# 27 대전광역시 대덕구 석봉로 38번길 55 대덕구보건소
# 94 광주광역시 서구 경열로 17번길 5
# 145 제주특별자치도 서귀포시 성산읍 일주동로 4120번길 7 동부소방서
# 189 충청남도 논산시 연무로 166번길 64 2주차장
# 196 제주특별자치도 제주시 선덕로 8길 12 설문대여성문화센터
# 236 대전광역시 중구 동서대로 1322번길 81
# 311 경기도 가평군 설악면 미사리로 540번길 51 가평휴게소  SK 가평주유소(춘천방향) 옆
# 362 광주광역시 광산구 광산로 68번길 13
# 365 제주특별자치도 서귀포시 동홍중앙로 7번길 19 자기주도학습지원센터
# 406 충청남도 홍성군 홍북읍 홍북로 453번길 50
# 433 대구광역시 달성군 구지면 창리로 11길 38
# 444 제주특별자치도 서귀포시 남성중로 153번길 15
# 446 제주특별자치도 서귀포시 효돈로 152번길 7
# 480 전라남도 여수시 학동 168(시청로 1번지)
# 535 서울특별시 영등포구 여의대방로 53길 22
# 537 서울특별시 강북구 도봉로 69길 29
# 578 서울특별시 서초구 남부순환로 340길 58
# 646 경상북도 구미시 봉곡로 15길 54
# 675 부산광역시 중구 대청로 116번길 3
# 724 인천광역시 중구 공항동로 296번길 69
# 739 강원도 철원군 철원읍 금학로 330번길 12
# 867 경기도 고양시 덕양구 화중로 104번길 13 입구 좌측
# 997 제주특별자치도 서귀포시 대정읍 추사로 36번길 1
# 1091 경기도 고양시 일산동구 중앙로 1275번길 64
# 1094 서울특별시 성동구 고산자로 26길 6
# 1111 경기도 이천시 마장면 덕이로 154번길 287-76
# 1116 경기도 성남시 분당구 야탑로 111번길 12-14
# 1117 경기도 성남시 분당구 성남대로 43번길 11
# 1214 충청북도 제천시 용두대로 11길 10
# 1234 서울특별시 양천구 남부순환로 88길 5-7
# 1392 서울특별시 마포구 상암동 월드컵북로 58길 15 지하4층 18
# 1408 부산광역시 사상구 낙동대로 1518번길 26
# 1422 경기도 여주시 여흥로 160번길 14
# 1428 강원도 인제군 남면 신남로 58번길 37
# 1435 경상북도 포항시 남구 오천읍 냉천로 320번길 4-11
# 1483 광주광역시 광산구 광산로 29번길 15
# 1487 제주특별자치도 서귀포시 보목로 64번길 155 보목하수처리장
# 1490 경상북도 경주시 유림로 5번길 85-15호
# 1496 전라남도 영광군 영광읍 천년로 13길 2-34 주차장
# 1501 경기도 동두천시 평화로 2169번길 21 (송내동)
# 1516 서울특별시 강남구 개포로 25길 32
# 1601 제주특별자치도 서귀포시 산록남로 1241번길 145(색달동)
# 1781 광주광역시 동구 필문대로 205번길 17
# 1834 광주광역시 동구 의재로 109번길 10
# 1836 서울특별시 금천구 독산로 36나길 21 B1 18
# 1853 서울특별시 강북구 오현로 21길 84 옥상 주차장
# 1873 서울특별시 강북구 도봉로 61길 46
# 2068 경기도 포천시 삼육사로 2186번길 11
# 2126 경기도 양주시 광적면 부흥로 618번길 303
# 2150 인천광역시 중구 영종해안북로 1204번길 105
# 2164 제주특별자치도 서귀포시 칠십리로 72번길 1
# 2178 서울특별시 양천구 중앙로 32길 1
# 2335 경기도 김포시 태장로 795번길 137
# 2348 서울특별시 영등포구 영신로 10길 15
# 2357 제주특별자치도 서귀포시 중앙로 101번길 52
# 2648 경기도 성남시 분당구 운중로 225번길 37
# 2655 경기도 수원시 권선구 경수대로 168번길 79
# 2730 경기도 시흥시 마유로 238번길 26
# 2784 제주특별자치도 서귀포시 신서귀로 51번길 38
# 2874 광주광역시 북구 첨단과기로 208번길 6
# 2926 광주광역시 서구 송암로 143번길 34
# 3012 광주광역시 광산구 왕버들로 322번길 6
# 3063 광주광역시 북구 첨단과기로 208번길 5
# 3173 인천광역시 미추홀구 아암대로 253번길 81
# 3358 인천광역시 서구 탁옥로 51번길 26
# 3490 제주특별자치도 서귀포시 도순남로 29번길 13
# 3507 제주특별자치도 서귀포시 남원읍 태신로 130번길 170
# 3534 대전광역시 유성구 대덕대로 989번길 111
# 3552 대전광역시 서구 갈마중로 30번길 15-5
# 3613 광주광역시 동구 충장로 5가 25
# 3627 대구광역시 달서구 송현로 8안길 6
# 3665 충청남도 아산시 모종북로 76번길 62
# 3676 충청남도 아산시 곡교천로 27번길 33
# 3850 충청북도 괴산군 청안면 조천로 2길 86
# 3889 울산광역시 남구 삼산로 35번길 25

In [1440]:
# 3415 전라북도 익산시 익산대로 52길27

In [1441]:
cidx1 = [27, 94, 145, 189, 196, 236, 311, 362, 365, 406, 433, 
         444, 446, 480, 535, 537, 578, 646, 675, 724, 739, 867, 
         997, 1091, 1094, 1111, 1116, 1117, 1214, 1234, 1392, 
         1408, 1422, 1428, 1435, 1483, 1487, 1490, 1496, 1501,
         1516, 1601, 1781, 1834, 1836, 1853, 1873, 2068, 2126,
         2150, 2164, 2178, 2335, 2348, 2357, 2648, 2655, 2730, 
         2784, 2874, 2926, 3012, 3063, 3173, 3358, 3490, 3507,
         3534, 3552, 3613, 3627, 3665, 3676, 3850, 3889]
len(cidx1)

75

In [1442]:
print(adr[3507])
check1.sub('\g<1>\g<2>', adr[3507], 1)

제주특별자치도 서귀포시 남원읍 태신로 130번길 170


'제주특별자치도 서귀포시 남원읍 태신로130번길 170'

In [1443]:
print(adr[3552])
check1.sub('\g<1>\g<2>', adr[3552], 1)

대전광역시 서구 갈마중로 30번길 15-5


'대전광역시 서구 갈마중로30번길 15-5'

In [1444]:
print(adr[480])
check1.sub('\g<1>\g<2>', adr[480], 1)

전라남도 여수시 학동 168(시청로 1번지)


'전라남도 여수시 학동 168(시청로1번지)'

In [1445]:
for i in cidx1:
    temp = adr[i]
    adr[i] = check1.sub('\g<1>\g<2>', temp, 1)

In [1446]:
adr[[480, 3507, 3552]]

array(['전라남도 여수시 학동 168(시청로1번지)', '제주특별자치도 서귀포시 남원읍 태신로130번길 170',
       '대전광역시 서구 갈마중로30번길 15-5'], dtype=object)

In [1447]:
adr[3415]

'전라북도 익산시 익산대로 52길27'

In [1448]:
print(adr[3415])
if adr[3415] == '전라북도 익산시 익산대로 52길27':
    adr[3415] = '전라북도 익산시 익산대로52길 27'
print(adr[3415])

전라북도 익산시 익산대로 52길27
전라북도 익산시 익산대로52길 27


###### -로 -길

In [1449]:
check12 = re.compile('([가-힣]+[0-9]*로) ([0-9]+길)')
for i, one in enumerate(adr):
    if check12.search(one):
        print(i, one)

###### -로 -번길

In [1450]:
check13 = re.compile('([가-힣]+[0-9]*로) ([0-9]+번길)')
for i, one in enumerate(adr):
    if check13.search(one):
        print(i, one)

###### -로 -가길

In [1451]:
check14 = re.compile('([가-힣]+[0-9]*로) ([0-9]+가길)')
for i, one in enumerate(adr):
    if check14.search(one):
        print(i, one)

###### -로 -나길

In [1452]:
check15 = re.compile('([가-힣]+[0-9]*로) ([0-9]+나길)')
for i, one in enumerate(adr):
    if check15.search(one):
        print(i, one)

###### -로 -다길

In [1453]:
check16 = re.compile('([가-힣]+[0-9]*로) ([0-9]+다길)')
for i, one in enumerate(adr):
    if check16.search(one):
        print(i, one)

2410 서울특별시 용산구 한강대로 62다길 42


In [1454]:
print(adr[2410])
if adr[2410] == '서울특별시 용산구 한강대로 62다길 42':
    adr[2410] = '서울특별시 용산구 한강대로62다길 42'
print(adr[2410])

서울특별시 용산구 한강대로 62다길 42
서울특별시 용산구 한강대로62다길 42


###### -로 -안길

In [1455]:
check17 = re.compile('([가-힣]+[0-9]*로) ([0-9]+안길)')
for i, one in enumerate(adr):
    if check17.search(one):
        print(i, one)

##### -길 체크

In [1456]:
check2 = re.compile('([가-힣]+[0-9]*길)([0-9]+)')
for i, one in enumerate(adr):
    if check2.search(one):
        print(i, one)

743 서울특별시 영등포구 신길1동 116-15
1479 경기도 양평군 개군면 하자포길33번길 8
1570 경상북도 포항시 북구 흥해읍 초곡길36번길 30
3424 경상남도 진주시 가좌길29번길 63
3673 경상북도 포항시 남구 구룡포읍 구룡포길117번길 28-8


###### -길 -번길

In [1457]:
check21 = re.compile('([가-힣]+[0-9]*길) ([0-9]+번길)')
for i, one in enumerate(adr):
    if check21.search(one):
        print(i, one)

2738 경상남도 진주시 망경남길 60번길 18


In [1458]:
print(adr[2738])
if adr[2738] == '경상남도 진주시 망경남길 60번길 18':
    adr[2738] = '경상남도 진주시 망경남길60번길 18'
print(adr[2738])

경상남도 진주시 망경남길 60번길 18
경상남도 진주시 망경남길60번길 18


##### -동 체크

In [1459]:
check3 = re.compile('([가-힣]+[0-9]*동)([0-9]+)')
for i, one in enumerate(adr):
    if check3.search(one):
        print(i, one)

389 충청북도 단양군 적성면 기동2길 32-37 (기동리)
439 전라북도 전주시 완산구 효자동3가 1712-10
443 전라북도 전주시 완산구 효자동2가 1234-3 5층
605 전라북도 전주시 덕진구 인후동1가 807-6
645 경상북도 구미시 인동14길 14
649 대구광역시 북구 노원동2가 214
687 경상남도 창원시 마산합포구 신마산시장길 33 (창포동3가)
854 경상남도 창원시 진해구 용재로 46  웅동2동 주민센터 주차장 내
982 서울특별시 종로구 인사동5길 29 지상주차장
1056 경상남도 창원시 장군동4가 3-6
1062 부산광역시 중구 중앙대로 2 (중앙동7가)
1135 충청북도 청주시 서원구 현도면 선동1길 9
1139 충청남도 당진시 무수동2길 25-21
1368 울산광역시 울주군 언양읍 언동3길 20-48
1426 강원도 삼척시 당저동1길 17-9
1748 인천광역시 중구 북성동1가 97-32
2421 전라북도 전주시 덕진구 송천동2가 1257
2423 전라북도 전주시 덕진구 송천동2가 1316
2616 경상남도 창원시 마산회원구 석전동4길 52
2854 인천광역시 중구 송월동2가 4
2862 전라북도 전주시 덕진구 가련산로 26-3 (송천동1가)
2970 경상남도 합천군 합천읍 교동1길 33
3069 경상남도 창원시 마산합포구 현동7길 27
3152 충청북도 영동군 황간면 원촌동1길 54-8
3285 전라북도 익산시 인화동2가 7
3453 전라남도 여수시 삼동3길 13
3891 경상남도 고성군 거류면 당동3길 5
3924 전라남도 여수시 학동2길 7
3987 전라남도 영암군 신북면 황금동1길 5-1
4050 충청북도 청주시 서원구 현도면 선동2길 232-9
4181 전라남도 여수시 관문동1길 39
4221 충청북도 단양군 적성면 기동2길 32-37


In [1460]:
# 이상없음

##### -가 체크

###### -동 -가

In [1461]:
check41 = re.compile('([가-힣]+[0-9]*동) ([0-9]+가)')
for i, one in enumerate(adr):
    if check41.search(one):
        print(i, one)

283 전라북도 전주시 덕진구 덕진동 2가 179-6
607 전라북도 전주시 덕진동 2가 694-1


In [1462]:
print(adr[283])
if adr[283] == '전라북도 전주시 덕진구 덕진동 2가 179-6':
    adr[283] = '전라북도 전주시 덕진구 덕진동2가 179-6'
print(adr[283])

전라북도 전주시 덕진구 덕진동 2가 179-6
전라북도 전주시 덕진구 덕진동2가 179-6


In [1463]:
print(adr[607])
if adr[607] == '전라북도 전주시 덕진동 2가 694-1':
    adr[607] = '전라북도 전주시 덕진동2가 694-1'
print(adr[607])

전라북도 전주시 덕진동 2가 694-1
전라북도 전주시 덕진동2가 694-1


###### -로 -가

In [1464]:
check42 = re.compile('([가-힣]+[0-9]*로) ([0-9]+가)')
for i, one in enumerate(adr):
    if check42.search(one):
        print(i, one)

### 주소 세분류

#### 빈칸 처리

In [1465]:
count = 0
for i, one in enumerate(adr):
    if '  ' in one:
        adr[i] = one.replace('  ', ' ')
        count += 1
count

205

#### 컴마 제거

In [1466]:
count = 0
for i, one in enumerate(adr):
    if ',' in one:
        adr[i] = one.replace(',', '')
        count += 1
count

0

In [1467]:
adr[858]

'경상남도 김해시 진영읍 하계로96번길 94-4 주유소 옆 주차장'

#### 시-구 같은 구분지역 처리

In [1468]:
split_dict = {' 고양시 덕양구 ':'$고양시-덕양구$', ' 고양시 일산동구 ':'$고양시-일산동구$', ' 고양시 일산서구 ':'$고양시-일산서구$', ' 성남시 분당구 ':'$성남시-분당구$', ' 성남시 수정구 ':'$성남시-수정구$', 
              ' 성남시 중원구 ':'$성남시-중원구$', ' 성남시 권선구 ':'$성남시-권선구$', ' 성남시 영통구 ':'$성남시-영통구$', ' 성남시 장안구 ':'$성남시-장안구$', ' 성남시 팔달구 ':'$성남시-팔달구$',
              ' 수원시 권선구 ':'$수원시-권선구$', ' 수원시 영통구 ':'$수원시-영통구$', ' 수원시 장안구 ':'$수원시-장안구$', ' 수원시 팔달구 ':'$수원시-팔달구$', ' 안산시 단원구 ':'$안산시-단원구$', 
              ' 안산시 상록구 ':'$안산시-상록구$', ' 안양시 동안구 ':'$안양시-동안구$', ' 안양시 만안구 ':'$안양시-만안구$', ' 용인시 기흥구 ':'$용인시-기흥구$', ' 용인시 수지구 ':'$용인시-수지구$', 
              ' 용인시 처인구 ':'$용인시-처인구$', ' 청주시 상당구 ':'$청주시-상당구$', ' 청주시 서원구 ':'$청주시-서원구$', ' 청주시 청원구 ':'$청주시-청원구$', ' 청주시 흥덕구 ':'$청주시-흥덕구$', 
              ' 천안시 동남구 ':'$천안시-동남구$', ' 천안시 서북구 ':'$천안시-서북구$', ' 전주시 덕진구 ':'$전주시-덕진구$', ' 전주시 완산구 ':'$전주시-완산구$', ' 창원시 마산합포구 ':'$창원시-마산합포구$', 
              ' 창원시 마산회원구 ':'$창원시-마산회원구$', ' 창원시 성산구 ':'$창원시-성산구$', ' 창원시 의창구 ':'$창원시-의창구$', ' 창원시 진해구 ':'$창원시-진해구$', ' 포항시 남구 ':'$포항시-남구$', 
              ' 포항시 북구 ':'$포항시-북구$'}

In [1469]:
len(split_dict)

36

In [1470]:
adr2 = []
for ad in tqdm(adr):
    for k, v in split_dict.items():
        ad = ad.replace(k, v)
    ad = ad.replace(' ', '$')
    ad = ad.replace('-', ' ')
    adr2.append(ad.split('$'))

100%|██████████| 4364/4364 [00:00<00:00, 69424.10it/s]


In [1471]:
adr2[:5]

[['강원도', '인제군', '상남면', '하남리', '339 7(양양방향)'],
 ['광주광역시', '북구', '첨단벤처로108번길', '9'],
 ['전라남도', '순천시', '해룡면', '남가리', '733 73'],
 ['전라남도', '구례군', '산동면', '산업로', '2319'],
 ['전라남도', '구례군', '산동면', '노고단로', '1068']]

#### 중복 주소명 확인

In [1472]:
adr3 = []
count = 0
for one in tqdm(adr2):
    temp = []
    for two in one:
        if two not in temp:
            temp.append(two)
        else:
            count += 1
            print(one)
    adr3.append(temp)

100%|██████████| 4364/4364 [00:00<00:00, 374428.61it/s]

['부산광역시', '기장군', '철마면', '철마면', '485']
['경상북도', '경산시', '경산시', '하양읍', '문화로', '36']
['경기도', '광주시', '중앙로', '103 5', '1층', '1층', '화장실', '옆']
['경기도', '시흥시', '장곡동', '724 32', '오른쪽', '주차장', '입구', '오른쪽', '끝부지']
['대구광역시', '달서구', '달서구', '공원순환로', '201']
['대구광역시', '달서구', '달서구', '송현동', '산56번지']
['대구광역시', '북구', '북구', '산격동', '산79 13']
['경상북도', '봉화군', '봉화군', '춘양면', '서벽리', '485']
['세종특별자치시', '다솜2로', '11', '정부세종청사', '종합안내실', '정문', '우측도보', '정부세종청사', '종합안내실', '정문', '우측도보']
['세종특별자치시', '다솜2로', '11', '정부세종청사', '종합안내실', '정문', '우측도보', '정부세종청사', '종합안내실', '정문', '우측도보']
['세종특별자치시', '다솜2로', '11', '정부세종청사', '종합안내실', '정문', '우측도보', '정부세종청사', '종합안내실', '정문', '우측도보']
['세종특별자치시', '다솜2로', '11', '정부세종청사', '종합안내실', '정문', '우측도보', '정부세종청사', '종합안내실', '정문', '우측도보']
['경상북도', '울릉군', '북면', '천부2길', '15', '울릉군', '북면사무소', '주차장']
['대구광역시', '동구', '효목동', '1042', '1042']
['경상남도', '김해시', '칠산로', '128', '128']
['경상북도', '영주시', '영주시', '문수면', '탄산리', '577 2']
['경상북도', '울릉군', '서면', '남양4길', '31', '울릉군', '서면사무소', '주차장']
['강원도', '원주시', '원주시', '황소마을

In [1473]:
count

18

#### 지역 구분으로 주소 분리
- 지역명 구분
    - 지역
    - 시군구
    - 읍면동
    - 리
    - 도로명

In [1474]:
ad1_p = re.compile('[가-힣]+도$|[가-힣]+시$')
ad2_p = re.compile('[가-힣]+[시군구]$')
ad3_p = re.compile('[가-힣0-9]+[읍면동가]$')
ad4_p = re.compile('[가-힣0-9]+리$')
ad5_p = re.compile('[가-힣0-9]+[로번길]$')

In [1475]:
s = '세종특별자치시'
ad1_p.search(s)

<re.Match object; span=(0, 7), match='세종특별자치시'>

In [1476]:
s = '제주특별자치도'
ad1_p.search(s)

<re.Match object; span=(0, 7), match='제주특별자치도'>

In [1477]:
s = '고양시 덕양구'
ad2_p.search(s)

<re.Match object; span=(4, 7), match='덕양구'>

In [1478]:
s = '강남구'
ad2_p.search(s)

<re.Match object; span=(0, 3), match='강남구'>

In [1479]:
s = '효자동3가'
ad3_p.search(s)

<re.Match object; span=(0, 5), match='효자동3가'>

In [1480]:
s = '서귀동'
ad3_p.search(s)

<re.Match object; span=(0, 3), match='서귀동'>

In [1481]:
s = '신길1동'
ad3_p.search(s)

<re.Match object; span=(0, 4), match='신길1동'>

In [1482]:
s = '죽림리'
ad4_p.search(s)

<re.Match object; span=(0, 3), match='죽림리'>

In [1483]:
s = '첨단벤처로108번길'
ad5_p.search(s)

<re.Match object; span=(0, 10), match='첨단벤처로108번길'>

In [1484]:
s = '노형9길'
ad5_p.search(s)

<re.Match object; span=(0, 4), match='노형9길'>

In [1485]:
s = '학동로'
ad5_p.search(s)

<re.Match object; span=(0, 3), match='학동로'>

In [1486]:
for i, adrs in tqdm(enumerate(adr3)):
    for j, one in enumerate(adrs):
        if j == 0:
            if ad1_p.search(one):
                pass
        elif j == 1:
            if ad2_p.search(one):
                pass
            else:
                adrs.insert(j, np.nan)
        elif j == 2:
            if ad3_p.search(one):
                pass
            else:
                adrs.insert(j, np.nan)
        elif j == 3:
            if ad4_p.search(one):
                pass
            else:
                adrs.insert(j, np.nan)


4364it [00:00, 161924.48it/s]


In [1487]:
# 5번째 구분에 '-리', 주소 존재 여부 체크
for i, one in enumerate(adr3):
    try:
        if ad4_p.search(one[4]):
            print(i, one)
    except:
        pass

In [1488]:
# 6번째 구분에 '-리', '-로, -길' 주소 존재 여부 체크
for i, one in enumerate(adr3):
    try:
        if ad4_p.search(one[5]) or ad5_p.search(one[5]):
            print(i, one)
    except:
        pass

169 ['경상남도', '통영시', nan, nan, '큰발개', '1길', '33', '마리나요트클럽', '앞']
405 ['충청남도', '천안시 서북구', nan, nan, '신당새터', '2길', '28', '카페', '옆']
509 ['경기도', '양평군', '양동면', '단석리', '513 2', '(여양3로', '384 27)']
901 ['인천광역시', '서구', nan, nan, '청라', '커낼로', '252(경서동)']
1034 ['경기도', '하남시', nan, nan, '미사강변', '한강로', '299', '지하주차장', '입구', '앞']
1041 ['대전광역시', '유성구', nan, nan, '테크노', '9로', '35', '입구', '국기계양대', '왼쪽']
1357 ['제주특별자치도', '제주시', nan, nan, '산천단', '동3길', '2', '북쪽주차장', '입구']
1825 ['경기도', '부천시', nan, nan, '원미구', '석천로', '188', '지상주차장', '2층', 'D구역', '창고', '오른쪽']
1918 ['경상북도', '상주시', nan, nan, '영남', '제일로', '1339']
1921 ['세종특별자치시', nan, nan, nan, '다솜', '3로', '66']
2090 ['충청남도', '당진시', '정미면', '천의리', '245 4', '천의삼거리', '주차장', '내']
2215 ['경기도', '이천시', '호법면', nan, '프리미엄', '아울렛로', '177 74']
2497 ['충청남도', '보령시', nan, nan, '갓바위', '1길', '10 30']
2735 ['울산광역시', '북구', nan, nan, '진장', '유통로', '64']
3760 ['부산광역시', '강서구', nan, nan, '명지오션시티', '13로', '12 11']


In [1489]:
# 169 ['경상남도', '통영시', nan, nan, '큰발개', '1길', '33', '마리나요트클럽', '앞']
# 405 ['충청남도', '천안시 서북구', nan, nan, '신당새터', '2길', '28', '카페', '옆']
# 901 ['인천광역시', '서구', nan, nan, '청라', '커낼로', '252(경서동)']
# 1034 ['경기도', '하남시', nan, nan, '미사강변', '한강로', '299', '지하주차장', '입구', '앞']
# 1041 ['대전광역시', '유성구', nan, nan, '테크노', '9로', '35', '입구', '국기계양대', '왼쪽']
# 1357 ['제주특별자치도', '제주시', nan, nan, '산천단', '동3길', '2', '북쪽주차장', '입구']
# 1918 ['경상북도', '상주시', nan, nan, '영남', '제일로', '1339']
# 1921 ['세종특별자치시', nan, nan, nan, '다솜', '3로', '66']
# 2215 ['경기도', '이천시', '호법면', nan, '프리미엄', '아울렛로', '177 74']
# 2497 ['충청남도', '보령시', nan, nan, '갓바위', '1길', '10 30']
# 2735 ['울산광역시', '북구', nan, nan, '진장', '유통로', '64']
# 3760 ['부산광역시', '강서구', nan, nan, '명지오션시티', '13로', '12 11']

In [1490]:
sidx1 = [169, 405, 901, 1034, 1041, 1357, 1918, 1921, 2215, 2497, 2735, 3760]
len(sidx1)

12

#### 하나씩 확인

In [1491]:
for i in tqdm(sidx1):
    adr3[i][4] = adr3[i][4] + adr3[i][5]
    adr3[i].pop(5)
    print(adr3[i])

100%|██████████| 12/12 [00:00<?, ?it/s]

['경상남도', '통영시', nan, nan, '큰발개1길', '33', '마리나요트클럽', '앞']
['충청남도', '천안시 서북구', nan, nan, '신당새터2길', '28', '카페', '옆']
['인천광역시', '서구', nan, nan, '청라커낼로', '252(경서동)']
['경기도', '하남시', nan, nan, '미사강변한강로', '299', '지하주차장', '입구', '앞']
['대전광역시', '유성구', nan, nan, '테크노9로', '35', '입구', '국기계양대', '왼쪽']
['제주특별자치도', '제주시', nan, nan, '산천단동3길', '2', '북쪽주차장', '입구']
['경상북도', '상주시', nan, nan, '영남제일로', '1339']
['세종특별자치시', nan, nan, nan, '다솜3로', '66']
['경기도', '이천시', '호법면', nan, '프리미엄아울렛로', '177 74']
['충청남도', '보령시', nan, nan, '갓바위1길', '10 30']
['울산광역시', '북구', nan, nan, '진장유통로', '64']
['부산광역시', '강서구', nan, nan, '명지오션시티13로', '12 11']


In [1492]:
adr3[2090]

['충청남도', '당진시', '정미면', '천의리', '245 4', '천의삼거리', '주차장', '내']

In [1493]:
adr3[2090][4] = '천의삼거리'
adr3[2090].insert(4, '4')
adr3[2090].insert(4, '245')
adr3[2090]

['충청남도', '당진시', '정미면', '천의리', '245', '4', '천의삼거리', '천의삼거리', '주차장', '내']

In [1494]:
adr3[4362]

['제주특별자치도', '제주시', nan, nan, '천수로', '72']

In [1495]:
ad5_p = re.compile('[가-힣0-9]+[로번길]$')

In [1496]:
# 5번째 구분에 '-로, -길' 주소 존재 여부 체크
count = 0
idx9 = []
for i, one in enumerate(adr3):
    try:
        if not ad5_p.search(one[4]):
            idx9.append(i)
            count += 1
            print(i, one)
    except:
        pass

0 ['강원도', '인제군', '상남면', '하남리', '339 7(양양방향)']
2 ['전라남도', '순천시', '해룡면', '남가리', '733 73']
5 ['전라남도', '장성군', '북하면', '약수리', '252 6']
6 ['전라남도', '장성군', '북하면', '신성리', '427 2']
7 ['전라북도', '부안군', '변산면', '격포리', '283 1']
8 ['전라남도', '영암군', '영암읍', '개신리', '484 47']
11 ['전라북도', '남원시', '산내면', '덕동리', '406']
12 ['전라북도', '정읍시', '내장동', nan, '52']
13 ['전라북도', '정읍시', '내장동', nan, '169 1']
14 ['전라북도', '무주군', '설천면', '삼공리', '411']
16 ['제주특별자치도', '제주시', '연동', nan, '2334 4']
17 ['제주특별자치도', '제주시', '조천읍', '함덕리', '906번지']
28 ['대전광역시', '대덕구', '신대동', nan, '169 1']
30 ['대전광역시', '서구', '만년동', nan, '74']
31 ['대전광역시', '서구', '복수동', nan, '613']
32 ['대전광역시', '서구', '월평동', nan, '1275']
33 ['대전광역시', '서구', '정림동', nan, '127 1']
34 ['대전광역시', '중구', '사정동', nan, '403번지']
36 ['경기도', '양주시', '옥정동', nan, '1039']
39 ['경기도', '부천시', '상동', nan, '549번지']
41 ['경기도', '안산시 상록구', '이동', nan, '620']
42 ['경기도', '하남시', '망월동', nan, '1 1']
45 ['인천광역시', '서구', '연희동', nan, '187 8']
46 ['인천광역시', '남동구', '논현동', nan, '762 3']
47 ['인천광역시', '남동구', '서창동', nan, '

In [1497]:
count

1535

In [1498]:
for i in tqdm(idx9):
    adr3[i].insert(4, np.nan)

100%|██████████| 1535/1535 [00:00<00:00, 769206.29it/s]


In [1499]:
idx9[:5]

[0, 2, 5, 6, 7]

In [1500]:
adr3[0]

['강원도', '인제군', '상남면', '하남리', nan, '339 7(양양방향)']

In [1501]:
adr3[7]

['전라북도', '부안군', '변산면', '격포리', nan, '283 1']

In [1502]:
adr3[858]

['경상남도', '김해시', '진영읍', nan, '하계로96번길', '94 4', '주유소', '옆', '주차장']

#### df 만들기

In [1503]:
sliced_adr = [one[:5] for one in adr3]

In [1504]:
sliced_adr_df = pd.DataFrame(sliced_adr, columns=['지역', '시군구', '읍면동', '리', '도로명'])
sliced_adr_df.head()

,지역,시군구,읍면동,리,도로명
0,강원도,인제군,상남면,하남리,NaN
1,광주광역시,북구,NaN,NaN,첨단벤처로108번길
2,전라남도,순천시,해룡면,남가리,NaN
3,전라남도,구례군,산동면,NaN,산업로
4,전라남도,구례군,산동면,NaN,노고단로


In [1505]:
sliced_adr_df.shape

(4364, 5)

In [1506]:
sliced_adr_df.iloc[858]

지역        경상남도
시군구        김해시
읍면동        진영읍
리          NaN
도로명    하계로96번길
Name: 858, dtype: object

In [1507]:
for one in sliced_adr_df['도로명'].unique():
    if not (('로' in str(one)) or ('길' in str(one)) or ('번길' in str(one))) and (one != np.nan):
        print(one)

nan
None


In [1508]:
len(df2['주소'])

4364

In [1509]:
df2.loc[858, '주소']

'경상남도 김해시 진영읍 하계로96번길 94-4 , 주유소 옆 주차장'

In [1510]:
sliced_adr_df['주소'] = df2['주소']
sliced_adr_df.head()

,지역,시군구,읍면동,리,도로명,주소
0,강원도,인제군,상남면,하남리,NaN,강원도 인제군 상남면 하남리 339-7(양양방향)
1,광주광역시,북구,NaN,NaN,첨단벤처로108번길,광주광역시 북구 첨단 벤처로 108번길 9
2,전라남도,순천시,해룡면,남가리,NaN,전라남도 순천시 해룡면 남가리 733-73
3,전라남도,구례군,산동면,NaN,산업로,전라남도 구례군 산동면 산업로 2319
4,전라남도,구례군,산동면,NaN,노고단로,전라남도 구례군 산동면 노고단로1068


In [1511]:
sliced_adr_df.tail()

,지역,시군구,읍면동,리,도로명,주소
4359,전라북도,장수군,장수읍,NaN,장천로,전라북도 장수군 장수읍 장천로 247
4360,인천광역시,서구,NaN,NaN,정서진1로,인천광역시 서구 정서진1로 41
4361,전라남도,장흥군,부산면,호계리,NaN,전라남도 장흥군 부산면 호계리 산215-2
4362,제주특별자치도,제주시,NaN,NaN,천수로,제주특별자치도 제주시 천수로 72
4363,경기도,용인시 처인구,NaN,NaN,동백죽전대로,경기도 용인시 처인구 동백죽전대로 61


In [1512]:
sliced_adr_df2 = sliced_adr_df.copy()
sliced_adr_df2.columns = ['ad1', 'ad2', 'ad3', 'ad4', 'ad5', '주소']
sliced_adr_df2 = sliced_adr_df2[['주소', 'ad1', 'ad2', 'ad3', 'ad4', 'ad5']]
sliced_adr_df2.head()

,주소,ad1,ad2,ad3,ad4,ad5
0,강원도 인제군 상남면 하남리 339-7(양양방향),강원도,인제군,상남면,하남리,NaN
1,광주광역시 북구 첨단 벤처로 108번길 9,광주광역시,북구,NaN,NaN,첨단벤처로108번길
2,전라남도 순천시 해룡면 남가리 733-73,전라남도,순천시,해룡면,남가리,NaN
3,전라남도 구례군 산동면 산업로 2319,전라남도,구례군,산동면,NaN,산업로
4,전라남도 구례군 산동면 노고단로1068,전라남도,구례군,산동면,NaN,노고단로


In [1513]:
sliced_adr_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4364 entries, 0 to 4363
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   주소      4364 non-null   object
 1   ad1     4364 non-null   object
 2   ad2     4333 non-null   object
 3   ad3     2947 non-null   object
 4   ad4     919 non-null    object
 5   ad5     2802 non-null   object
dtypes: object(6)
memory usage: 204.7+ KB


In [1514]:
sliced_adr_df2[sliced_adr_df2['ad5'].notnull() == True].head()

,주소,ad1,ad2,ad3,ad4,ad5
1,광주광역시 북구 첨단 벤처로 108번길 9,광주광역시,북구,NaN,NaN,첨단벤처로108번길
3,전라남도 구례군 산동면 산업로 2319,전라남도,구례군,산동면,NaN,산업로
4,전라남도 구례군 산동면 노고단로1068,전라남도,구례군,산동면,NaN,노고단로
9,전라남도 무안군 망운면 공항로 970-260,전라남도,무안군,망운면,NaN,공항로
10,전라남도 무안군 삼향읍 오룡1길 (E주차장-전라남도관광안내소),전라남도,무안군,삼향읍,NaN,오룡1길


In [1515]:
new_adr_idx = sliced_adr_df2[sliced_adr_df2['ad5'].notnull() == True].index
len(new_adr_idx)

2802

In [1516]:
new_adr_idx = list(new_adr_idx)
new_adr_idx[:5]

[1, 3, 4, 9, 10]

In [1517]:
sort_idx = ['old']*4364

In [1518]:
for i in new_adr_idx:
    sort_idx[i] = 'new'

In [1519]:
sort_idx[:10]

['old', 'new', 'old', 'new', 'new', 'old', 'old', 'old', 'old', 'new']

In [1520]:
sliced_adr_df2['주소분류'] = sort_idx
sliced_adr_df2.head()

,주소,ad1,ad2,ad3,ad4,ad5,주소분류
0,강원도 인제군 상남면 하남리 339-7(양양방향),강원도,인제군,상남면,하남리,NaN,old
1,광주광역시 북구 첨단 벤처로 108번길 9,광주광역시,북구,NaN,NaN,첨단벤처로108번길,new
2,전라남도 순천시 해룡면 남가리 733-73,전라남도,순천시,해룡면,남가리,NaN,old
3,전라남도 구례군 산동면 산업로 2319,전라남도,구례군,산동면,NaN,산업로,new
4,전라남도 구례군 산동면 노고단로1068,전라남도,구례군,산동면,NaN,노고단로,new


In [1521]:
sliced_adr_df2.loc[858]

주소      경상남도 김해시 진영읍 하계로96번길 94-4 , 주유소 옆 주차장
ad1                                      경상남도
ad2                                       김해시
ad3                                       진영읍
ad4                                       NaN
ad5                                   하계로96번길
주소분류                                      new
Name: 858, dtype: object

## 파일로 출력

In [1522]:
sliced_adr_df2.to_excel('분석/2020-22년_급속충전_주소_분류.xlsx', encoding='utf-8', index=False)

# 코드 마지막